In [4]:
import os

GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
GOOGLE_API_KEY
import google.generativeai as genai
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)
model = genai.GenerativeModel('gemini-pro')

RetryError: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv4:142.250.194.10:443: tcp handshaker shutdown

In [37]:
from PyPDF2 import PdfReader
def get_pdf_txt(pdf_docs):
    text=""
    # for pdf in pdf_docs:
    pdf_reader=PdfReader(pdf_docs)
    print(len(pdf_reader.pages))
    for page in pdf_reader.pages:
        text=text+page.extract_text()
    return text


In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
def get_txt_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
    return vector_store

In [61]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    print(prompt)
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [59]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    # response = chain.invoke(user_question)
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    # st.write("Reply: ", response["output_text"])

In [62]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper,VectorstoreIndexCreator
text=get_pdf_txt("./story.pdf")

text_chunks=get_txt_chunks(text)
print(text_chunks)
vectorStore=get_vector_store(text_chunks)
print(vectorStore)
user_input("tell the sory A BOX FULL OF KISSES ")



44
['Short Stories  for Children  \nfor  \nSpoken English Program  \n \n \n \n \n  \n \nSpoken English: Short Stories  \n2 \n  \nShort Stories for Children  \nLEVEL 1: STORIES FOR PRIMARY SCHOOL CHILDREN  ................................ ................................ ..............  5 \nTHE WIND AND THE SUN  ................................ ................................ ................................ ....................  5 \nTHE VILLAGER AND THE SPECTACLES  ................................ ................................ ................................ . 5 \nAS YOU SOW, SO SHALL YOU REAP  ................................ ................................ ................................ ..... 6 \nTHE FARMER AND HIS SONS  ................................ ................................ ................................ ...............  6 \nBIRBAL THE WISE  ................................ ................................ ................................ .......................